In [1]:
import numpy as np
from scipy.stats import norm
from scipy.special import comb
import matplotlib.pyplot as plt
import pandas as pd
import random

In [33]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile('JP_equity.xlsx')

# Running the provided code
# Dictionary to hold data for each stock
market_value_data = {}
stock_data = {}
monthly_returns_data = {}
pb_ratio_data = {}

# Iterate through each sheet (representing each company)
for sheet_name in xls.sheet_names:
    # Read the specific sheet
    df = pd.read_excel(xls, sheet_name)

    # Filter rows where date is on or after 2011-06-01
    filtered_df = df[df['Date'] >= '2011-06-01']
    
    # Set the 'Date' column as the index
    filtered_df.set_index('Date', inplace=True)

    # Extract the 'mom_return' column
    mom_returns = filtered_df["mom_return"]
    stock_data[sheet_name] = mom_returns

    market_values = filtered_df["market_value(millions)"]
    market_value_data[sheet_name] = market_values
    
    monthly_returns = (filtered_df['close'] - filtered_df['current_price']) / filtered_df['current_price']
    monthly_returns_data[sheet_name] = monthly_returns
    
    pb_ratios = filtered_df['PB_ratio']
    pb_ratio_data[sheet_name] = pb_ratios

# Creating consolidated dataframes
consolidated_data = pd.DataFrame(stock_data)
consolidated_monthly_returns = pd.DataFrame(monthly_returns_data)
consolidated_pb_ratios = pd.DataFrame(pb_ratio_data)
consolidated_market_value = pd.DataFrame(market_value_data)

# Displaying a snippet of each consolidated dataframe
consolidated_data_snippet = consolidated_data.head()
consolidated_monthly_returns_snippet = consolidated_monthly_returns.head()
consolidated_pb_ratios_snippet = consolidated_pb_ratios.head()
consolidated_market_value_snippet = consolidated_market_value.head()

(consolidated_data_snippet, consolidated_monthly_returns_snippet, consolidated_pb_ratios_snippet, consolidated_market_value_snippet)


(                6857      6971      9984      6954      9983      6762
 Date                                                                  
 2011-06-01 -0.196391  0.172176  0.326445  0.231755 -0.124260 -0.129196
 2011-07-01 -0.211569  0.057069  0.172147  0.310480  0.003094 -0.153551
 2011-08-01 -0.140527  0.155462  0.248445  0.619734  0.181347 -0.093182
 2011-09-01 -0.405780 -0.112801 -0.074698  0.187206  0.231293 -0.282492
 2011-10-01 -0.446114 -0.185554 -0.115058 -0.070386  0.328585 -0.405223,
                 6857      6971      9984      6954      9983      6762
 Date                                                                  
 2011-06-01 -0.027741 -0.042303 -0.036566  0.071257  0.095439  0.030374
 2011-07-01 -0.069293  0.012270 -0.006601  0.091928  0.054742 -0.095238
 2011-08-01 -0.279562 -0.151515 -0.160465 -0.136208  0.058480 -0.162907
 2011-09-01 -0.140831 -0.065714 -0.092996 -0.141838 -0.033840 -0.181737
 2011-10-01  0.100236  0.071865  0.129145  0.193906  0.017155  

In [34]:
# Step 1: Ranking stocks by mom_return and dividing into three groups for each date
# mom
ranked_data = consolidated_data.rank(axis=1, method='first', ascending=False)  

# value
#ranked_data = consolidated_pb_ratios.rank(axis=1, ascending=True)

# Dividing into top, middle, and bottom thirds
top_third = ranked_data[ranked_data <= len(ranked_data.columns) / 3]
middle_third = ranked_data[(ranked_data > len(ranked_data.columns) / 3) & (ranked_data <= len(ranked_data.columns) * 2 / 3)]
bottom_third = ranked_data[ranked_data > len(ranked_data.columns) * 2 / 3]

# Step 2: Constructing portfolios
# Function to calculate weighted returns for a group
def calculate_weighted_returns(group, returns_data, market_value_data):
    weights = market_value_data[group.notna()]
    weighted_returns = (weights.div(weights.sum(axis=1), axis=0) * returns_data[group.notna()]).sum(axis=1)
    return weighted_returns

# Calculating weighted returns for each group
portfolio_top = calculate_weighted_returns(top_third, consolidated_monthly_returns, consolidated_market_value)
portfolio_middle = calculate_weighted_returns(middle_third, consolidated_monthly_returns, consolidated_market_value)
portfolio_bottom = calculate_weighted_returns(bottom_third, consolidated_monthly_returns, consolidated_market_value)

# Combining the portfolio returns into a single DataFrame
portfolios_returns = pd.DataFrame({
    'Top Third Portfolio': portfolio_top,
    'Middle Third Portfolio': portfolio_middle,
    'Bottom Third Portfolio': portfolio_bottom
})

portfolios_returns.head()  # Displaying the first few rows of the portfolio returns


,Top Third Portfolio,Middle Third Portfolio,Bottom Third Portfolio
Date,,,
2011-06-01,0.016523,0.022264,0.010643
2011-07-01,0.043853,0.032614,-0.086264
2011-08-01,-0.147871,-0.039331,-0.199391
2011-09-01,-0.102537,-0.083990,-0.168510
2011-10-01,0.135986,0.110890,0.167106


In [35]:
portfolios_returns.to_excel("JP_port_mom.xlsx")